# Czech Wikipedia in the covid era: Pageviews

This notebook shows data about pageviews for Czech Wikipedia pages during years 2020 and 2019.

Created by Martin Urbanec, Wikimedia Czech Republic. WIP notebook, do not rely on (yet).

In [2]:
from wmfdata import spark
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Configuration

In [6]:
SNAPSHOT = '2021-05'
PROJECT='cs.wikipedia'
DBNAME = 'cswiki'

## Helper methods

In [5]:
def merge_dataframes(file_2019, file_2020, suffixes=('_2019', '_2020')):
    df_2019 = pd.read_csv(file_2019, sep='\t')
    df_2019['date'] = df_2019.date.str.replace('2019-', 'year-')
    df_2019.set_index('date', inplace=True)

    df_2020 = pd.read_csv(file_2020, sep='\t')
    df_2020['date'] = df_2020.date.str.replace('2020-', 'year-')
    df_2020.set_index('date', inplace=True)

    df = df_2019.merge(df_2020, left_index=True, right_index=True, suffixes=suffixes)
    df.reset_index(inplace=True)
    df['date'] = df.date.str.replace('year-', '2020-')
    df.set_index('date', inplace=True)
    
    return df

## Total pageviews at Czech Wikipedia
Daily pageviews of all Czech Wikipedia pages during 2020 and 2019.

In [17]:
def get_views_per_year(year):
    return spark.run('''
    SELECT
        CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) AS `date`,
        SUM(view_count) AS views
    FROM wmf.projectview_hourly
    WHERE
            year={year}
        AND project="{project}"
    GROUP BY `date`
    ORDER BY `date`
    '''.format(
        project=PROJECT,
        year=year
    ))

In [18]:
df = get_views_per_year(2020)
df.to_csv('data/cswiki_pageviews_2020.tsv', sep='\t', index=False)
df.head()

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,date,views
0,2020-01-01,3586942
1,2020-01-02,3534903
2,2020-01-03,3483584
3,2020-01-04,3783583
4,2020-01-05,4026342


In [19]:
df = get_views_per_year(2019)
df.to_csv('data/cswiki_pageviews_2019.tsv', sep='\t', index=False)
df.head()

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,date,views
0,2019-01-01,3338366
1,2019-01-02,3454958
2,2019-01-03,3420861
3,2019-01-04,3221445
4,2019-01-05,3802748


In [20]:
df = merge_dataframes('data/cswiki_pageviews_2019.tsv', 'data/cswiki_pageviews_2020.tsv')
df.to_csv('data/cswiki_pageviews_2019_2020.tsv', sep='\t')
df.head()

,views_2019,views_2020
date,,
2020-01-01,3338366,3586942
2020-01-02,3454958,3534903
2020-01-03,3420861,3483584
2020-01-04,3221445,3783583
2020-01-05,3802748,4026342


## Pageviews difference -- helper methods

This section has helper methods to calculate pageview differences below.

In [19]:
def calculate_pageviews_difference(month, day):
    fillPageviewsQueryTemplate = '''
    INSERT INTO urbanecm.blogpost_covid_and_cswiki_pageviews

    SELECT
        year,
        month,
        day,
        page_title,
        SUM(view_count) AS views_{year}
    FROM wmf.pageview_hourly
    WHERE
            year = {year}
        AND month = {month}
        AND day = {day}
        AND project = '{project}'
        
        -- user traffic only
        AND agent_type='user'
        
        -- exclude false positive
        AND page_title != 'Venuše_(planeta)'
    GROUP BY
        year,
        month,
        day,
        page_title
    '''
    
    queries = [
        'DROP TABLE IF EXISTS urbanecm.blogpost_covid_and_cswiki_pageviews',
        '''
        CREATE TABLE urbanecm.blogpost_covid_and_cswiki_pageviews(
            year bigint,
            month bigint,
            day bigint,
            page_title string,
            views bigint
        )
        ''',
        fillPageviewsQueryTemplate.format(project=PROJECT, year=2019, month=month, day=day),
        fillPageviewsQueryTemplate.format(project=PROJECT, year=2020, month=month, day=day),
        '''
        SELECT
            year,
            month,
            day,
            SUM(views)
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        GROUP BY
            year,
            month,
            day
        '''
    ]

    # fill temp pageviews table -- urbanecm.blogpost_covid_and_cswiki_pageviews
    spark.run(queries)
    
    # return data
    return spark.run('''
    WITH pv_2020 AS (
        SELECT
            month, day, page_title, views AS views_2020
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        WHERE
            year=2020
    ),
    pv_2019 AS (
        SELECT
            month, day, page_title, views AS views_2019
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        WHERE
            year=2019
    )

    SELECT
        pv_2020.month,
        pv_2020.day,
        pv_2020.page_title,
        views_2020,
        views_2019,
        views_2020 - views_2019 AS views_difference
    FROM pv_2020
    JOIN pv_2019 ON
        pv_2020.month=pv_2019.month
        AND pv_2020.day=pv_2019.day
        AND pv_2020.page_title=pv_2019.page_title
    ORDER BY views_difference DESC
    LIMIT 20
    ''')

In [20]:
def calculate_pageviews_difference_month(month):
    fillPageviewsQueryTemplate = '''
    INSERT INTO urbanecm.blogpost_covid_and_cswiki_pageviews

    SELECT
        year,
        month,
        page_title,
        SUM(view_count) AS views_{year}
    FROM wmf.pageview_hourly
    WHERE
            year = {year}
        AND month = {month}
        AND project = '{project}'
        
        -- user traffic only
        AND agent_type='user'
        
        -- exclude false positive articles
        AND page_title != 'Venuše_(planeta)'
    GROUP BY
        year,
        month,
        page_title
    '''
    
    queries = [
        'DROP TABLE IF EXISTS urbanecm.blogpost_covid_and_cswiki_pageviews',
        '''
        CREATE TABLE urbanecm.blogpost_covid_and_cswiki_pageviews(
            year bigint,
            month bigint,
            page_title string,
            views bigint
        )
        ''',
        fillPageviewsQueryTemplate.format(project=PROJECT, year=2019, month=month),
        fillPageviewsQueryTemplate.format(project=PROJECT, year=2020, month=month),
        '''
        SELECT
            year,
            month,
            SUM(views)
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        GROUP BY
            year,
            month
        '''
    ]

    # fill temp pageviews table -- urbanecm.blogpost_covid_and_cswiki_pageviews
    spark.run(queries)
    
    # return data
    return spark.run('''
    WITH pv_2020 AS (
        SELECT
            month, page_title, views AS views_2020
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        WHERE
            year=2020
    ),
    pv_2019 AS (
        SELECT
            month, page_title, views AS views_2019
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        WHERE
            year=2019
    )

    SELECT
        pv_2020.month,
        pv_2020.page_title,
        views_2020,
        views_2019,
        views_2020 - views_2019 AS views_difference
    FROM pv_2020
    JOIN pv_2019 ON
        pv_2020.month=pv_2019.month
        AND pv_2020.page_title=pv_2019.page_title
    ORDER BY views_difference DESC
    LIMIT 20
    ''')

## Pageviews -- biggest difference

In this section, I at months with higher projectviews in 2020 than in 2019 in more details, to figure out if the pandemic is the cause of the increased page views. For each of the months, I calculate pageviews of all articles in 2020 and 2019 (for that month only) and show 20 with the top difference.

An alternative could be to use [Topviews](https://pageviews.toolforge.org/topviews), but that tool looks at top visited articles in any given month, without any regard to the previous year(s).

### March

In [21]:
df = calculate_pageviews_difference_month(month=3)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_March.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,3,Koronavirus,343732,239,343493
1,3,Španělská_chřipka,237451,1701,235750
2,3,Speciální:Hledání,1186872,960304,226568
3,3,Morfologie_květu,162610,1293,161317
4,3,Adam_Vojtěch,163364,3185,160179
5,3,SARS,151115,1507,149608
6,3,Pandemie,122521,656,121865
7,3,Roman_Prymula,98284,231,98053
8,3,Itálie,106522,14841,91681
9,3,Nouzový_stav,85974,217,85757


### April

In [22]:
df = calculate_pageviews_difference_month(month=4)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_April.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,4,Speciální:Hledání,1242920,885467,357453
1,4,Emanuel_Moravec,135184,2209,132975
2,4,-,288321,208763,79558
3,4,Adam_Vojtěch,72599,1689,70910
4,4,Španělská_chřipka,64491,1058,63433
5,4,Koronavirus,53295,219,53076
6,4,Kim_Čong-un,51866,4015,47851
7,4,Seznam_států_světa_podle_počtu_obyvatel,57879,12175,45704
8,4,Eva_Burešová,45936,2037,43899
9,4,Ivan_Stěpanovič_Koněv,43092,1187,41905


### July

In [23]:
df = calculate_pageviews_difference_month(month=7)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_July.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,7,Kateřina_II._Veliká,102341,2158,100183
1,7,Mistrovství_Evropy_ve_fotbale_2004,34740,1702,33038
2,7,Miloš_Jakeš,41294,10254,31040
3,7,Jiří_Procházka_(sportovec),28511,1268,27243
4,7,Ludvík_XIV.,30446,3515,26931
5,7,Petr_III._Ruský,27354,706,26648
6,7,Dýmějový_mor,27960,1696,26264
7,7,Jan_Skopeček,27075,2518,24557
8,7,Mistrovství_Evropy_ve_fotbale_2000,24521,498,24023
9,7,Čeština,28343,5763,22580


### October

In [24]:
df = calculate_pageviews_difference_month(month=10)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_October.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,10,Roman_Prymula,119620,304,119316
1,10,Alois_Rašín,58505,3966,54539
2,10,Tvoje_tvář_má_známý_hlas_(7._řada),53999,156,53843
3,10,Španělská_chřipka,52137,2257,49880
4,10,Náhorní_Karabach,47047,1388,45659
5,10,Jaroslav_Flegr,45587,682,44905
6,10,Martin_Havelka,45229,923,44306
7,10,App_Store,41861,1322,40539
8,10,Vitamín_D,46260,8019,38241
9,10,Viktorie_(britská_královna),42853,9819,33034


### November

In [25]:
df = calculate_pageviews_difference_month(month=11)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_November.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,11,Joe_Biden,187575,1046,186529
1,11,Speciální:Hledání,1042332,888121,154211
2,11,Donald_Trump,105739,7055,98684
3,11,Volby_prezidenta_USA_2020,87038,1446,85592
4,11,Volba_prezidenta_Spojených_států_amerických,85687,1053,84634
5,11,Diego_Maradona,86747,2423,84324
6,11,Alžběta_II.,113503,32817,80686
7,11,Viktorie_(britská_královna),81170,8989,72181
8,11,Seznam_prezidentů_Spojených_států_amerických,70954,9454,61500
9,11,Spojené_státy_americké,89908,30595,59313


### December

In [26]:
df = calculate_pageviews_difference_month(month=12)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_December.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,12,Český_státní_svátek,128175,8355,119820
1,12,Jan_Mikolášek,55371,406,54965
2,12,Alžběta_II.,87525,34739,52786
3,12,App_Store,47912,439,47473
4,12,"Margaret,_hraběnka_Snowdon",52353,10060,42293
5,12,Ladislav_Mrkvička,39246,1113,38133
6,12,Princezna_Diana,39991,6341,33650
7,12,Zodiac_(vrah),31804,2339,29465
8,12,Vlastimil_Brodský,37000,7776,29224
9,12,"Princ_Philip,_vévoda_z_Edinburghu",39125,11278,27847


Exception in thread Thread-32:
Traceback (most recent call last):
  File "/home/urbanecm/.conda/envs/2021-04-07T23.21.00_urbanecm/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/urbanecm/.conda/envs/2021-04-07T23.21.00_urbanecm/lib/python3.7/threading.py", line 1177, in run
    self.function(*self.args, **self.kwargs)
TypeError: stop_session() missing 1 required positional argument: 'session'

